<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/456_TPRO_RiskAnalysis_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## Why This Makes the Agent Better Than Most LLM-Based Systems

Most agents:

* collapse signals into a score immediately
* hide reasoning in prompts
* mix explanation and decision
* lose audit trails

Your system:

* builds a case
* records the case
* then decides *later*

That’s why it’s trustworthy.

---

# Risk Analysis Node — Structured Reasoning Without Scoring

## What This Node Does (High-Level)

The `risk_analysis_node` is where the orchestrator **builds understanding** — but deliberately **does not decide yet**.

Its job is to:

* collect evidence
* analyze risk dimensions independently
* preserve structure
* identify *why* a vendor might be risky

This separation is critical:
👉 **analysis first, judgment later**

---

## 1. Why This Node Exists as Its Own Phase

In real risk teams, analysis and scoring are separate steps.

This node mirrors that reality by:

* avoiding thresholds
* avoiding aggregation
* avoiding escalation logic
* avoiding business decisions

Instead, it creates a **fact pattern** per vendor.

That design choice dramatically improves:

* explainability
* auditability
* testability
* future model evolution

---

## 2. Dependency Injection via Utilities (Excellent Choice)

```python
from agents.third_party_risk_orchestrator.utilities.risk_analysis import (
    analyze_control_compliance,
    analyze_external_signals,
    analyze_performance_metrics,
    detect_risk_drift,
    identify_risk_drivers
)
```

This is *textbook orchestration*.

The node:

* coordinates
* sequences
* aggregates

The utilities:

* compute
* specialize
* remain testable in isolation

This prevents:

* monolithic logic
* hidden coupling
* untestable reasoning

It also lets you improve any analysis dimension without touching orchestration.

---

## 3. Defensive State Access (Quietly Very Important)

```python
third_parties = state.get("third_parties", [])
...
risk_domain_lookup = state.get("risk_domain_lookup", {})
```

You never assume the state is perfect.

This allows:

* graceful failure
* partial testing
* future node reuse
* clearer error messages

Combined with early exits, this avoids cascading failures later in scoring.

---

## 4. Explicit Precondition Enforcement

```python
if not third_parties:
    return {
        "errors": errors + ["risk_analysis_node: third_parties required"]
    }
```

This is not a guard clause — it’s a **quality gate**.

You’re enforcing:

> “No analysis without vendors.”

That may seem obvious, but many systems fail here and produce empty or misleading outputs.

You’ve chosen correctness over convenience.

---

## 5. Vendor-by-Vendor Isolation (Critical for Accountability)

```python
for vendor in third_parties:
```

Each vendor is analyzed **independently**.

Why this matters:

* failures don’t contaminate other vendors
* explanations stay scoped
* future parallelization is trivial
* drift detection stays clean

This design also makes it easy to answer:

> “Why is *this* vendor risky?”

without cross-talk.

---

## 6. Dimension-by-Dimension Analysis (No Cross-Pollution)

For each vendor, you run **five distinct analyses**:

### 1️⃣ Control Compliance

```python
analyze_control_compliance(...)
```

Evaluates:

* required controls per domain
* evidence status
* confidence gaps

This answers:

> “Is the vendor meeting expected safeguards?”

---

### 2️⃣ External Signals

```python
analyze_external_signals(...)
```

Evaluates:

* incidents
* regulatory actions
* negative media
* severity and recency

This answers:

> “Is the outside world telling us something changed?”

---

### 3️⃣ Performance Metrics

```python
analyze_performance_metrics(...)
```

Evaluates:

* SLA compliance
* incident frequency
* responsiveness
* satisfaction

This answers:

> “Is the vendor operationally reliable?”

---

### 4️⃣ Risk Drift Detection (Major Differentiator)

```python
detect_risk_drift(...)
```

This compares **past vs present**.

It answers:

> “Is risk increasing, decreasing, or stable — and why?”

Most agents *don’t model time*.
You explicitly do.

---

### 5️⃣ Risk Driver Identification (Narrative Layer)

```python
identify_risk_drivers(...)
```

This is where structure becomes explanation.

Instead of raw metrics, you generate **human-readable drivers**:

* “Expired SOC2”
* “Recent high-severity incident”
* “Repeated SLA breaches”

This is the bridge between:

* data
* decision
* executive communication

---

## 7. Clean Aggregation Without Scoring

```python
vendor_risk_analysis[vendor_id] = {
    ...
}
```

Notice what’s missing:

* no scores
* no thresholds
* no labels
* no escalation flags

This is intentional and *very good*.

You are preserving **raw analytical signal**, not collapsing it prematurely.

That makes downstream scoring:

* testable
* adjustable
* explainable

---

## 8. Drift Stored Separately (Subtle but Smart)

```python
risk_drift_detection[vendor_id] = drift_info
```

Drift is not mixed into current analysis.

Why that matters:

* avoids double counting
* allows independent KPI tracking
* supports early warning dashboards
* keeps historical reasoning intact

This separation is rare and excellent.

---

## 9. Failure Handling Philosophy

```python
except Exception as e:
    return {
        "errors": errors + [f"risk_analysis_node: Unexpected error - {str(e)}"]
    }
```

You do not:

* crash silently
* partially return results
* hide the failure

You fail *explicitly* and *traceably*.

That’s exactly what you want in a risk system.

---

## 10. Why This Node Is High Quality

This node:

* reasons without deciding
* structures without summarizing
* isolates concerns
* preserves evidence
* enables explanation later

In other words, it behaves like a **risk analyst**, not a judge.




In [ ]:
"""Risk analysis utilities for Third-Party Risk Orchestrator

This module contains utilities to analyze risk across all dimensions:
- Control compliance analysis
- External signals analysis
- Performance metrics analysis
- Risk drift detection
- Risk driver identification

All utilities are pure functions, independently testable.

Following MVP-first approach: Rule-based analysis, no LLM dependencies.
"""

from typing import List, Dict, Any, Optional
from datetime import datetime, date
from collections import defaultdict


def analyze_control_compliance(
    vendor_id: str,
    vendor_controls: List[Dict[str, Any]],
    risk_domains: List[Dict[str, Any]],
    risk_domain_lookup: Dict[str, Dict[str, Any]]
) -> Dict[str, Dict[str, Any]]:
    """
    Analyze control compliance for a vendor across all risk domains.

    Args:
        vendor_id: Vendor to analyze
        vendor_controls: All vendor controls
        risk_domains: All risk domain definitions
        risk_domain_lookup: Fast lookup for risk domains

    Returns:
        Dictionary mapping risk_domain to compliance analysis
        Structure:
        {
            "Information Security": {
                "status": "partial" | "active" | "expired" | "missing",
                "score": 0-100,
                "controls_analyzed": 2,
                "controls_active": 1,
                "controls_expired": 1,
                "controls_partial": 0,
                "controls_missing": 1,
                "missing_controls": ["SOC2"],
                "required_controls": ["SOC2", "Encryption", "Access Controls"],
                "controls_detail": [...]
            }
        }
    """
    # Filter controls for this vendor
    vendor_control_list = [c for c in vendor_controls if c.get("vendor_id") == vendor_id]

    # Group controls by risk domain
    domain_controls = defaultdict(list)
    for control in vendor_control_list:
        domain = control.get("risk_domain")
        if domain:
            domain_controls[domain].append(control)

    # Analyze each risk domain
    domain_analysis = {}

    for domain_def in risk_domains:
        domain_name = domain_def.get("risk_domain")
        required_controls = domain_def.get("required_controls", [])
        domain_control_list = domain_controls.get(domain_name, [])

        # Count control statuses
        controls_active = sum(1 for c in domain_control_list if c.get("status") == "active")
        controls_expired = sum(1 for c in domain_control_list if c.get("status") == "expired")
        controls_partial = sum(1 for c in domain_control_list if c.get("status") == "partial")

        # Find which required controls are present
        present_controls = {c.get("control") for c in domain_control_list}
        missing_controls = [c for c in required_controls if c not in present_controls]

        # Calculate compliance score
        # Active = 100%, Partial = 50%, Expired = 0%, Missing = 0%
        total_required = len(required_controls)
        if total_required == 0:
            score = 100.0
        else:
            score = (
                (controls_active * 100.0) +
                (controls_partial * 50.0) +
                (controls_expired * 0.0)
            ) / total_required

        # Determine overall status
        if controls_expired > 0 or len(missing_controls) > 0:
            if controls_active > 0 or controls_partial > 0:
                status = "partial"
            else:
                status = "missing"
        elif controls_partial > 0:
            status = "partial"
        elif controls_active == total_required and len(missing_controls) == 0:
            status = "active"
        else:
            status = "partial"

        domain_analysis[domain_name] = {
            "status": status,
            "score": round(score, 2),
            "controls_analyzed": len(domain_control_list),
            "controls_active": controls_active,
            "controls_expired": controls_expired,
            "controls_partial": controls_partial,
            "controls_missing": len(missing_controls),
            "missing_controls": missing_controls,
            "required_controls": required_controls,
            "controls_detail": domain_control_list
        }

    return domain_analysis


def analyze_external_signals(
    vendor_id: str,
    external_signals: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Analyze external risk signals for a vendor.

    Args:
        vendor_id: Vendor to analyze
        external_signals: All external signals

    Returns:
        Dictionary with signal analysis
        Structure:
        {
            "total_signals": 2,
            "high_severity_count": 1,
            "medium_severity_count": 1,
            "low_severity_count": 0,
            "signal_types": {
                "security_incident": 1,
                "regulatory_notice": 1
            },
            "recent_signals": [...],  # Signals from last 30 days
            "high_priority_signals": [...],  # High severity signals
            "signal_impact_score": 0-100  # Weighted impact score
        }
    """
    # Filter signals for this vendor
    vendor_signals = [s for s in external_signals if s.get("vendor_id") == vendor_id]

    if not vendor_signals:
        return {
            "total_signals": 0,
            "high_severity_count": 0,
            "medium_severity_count": 0,
            "low_severity_count": 0,
            "signal_types": {},
            "recent_signals": [],
            "high_priority_signals": [],
            "signal_impact_score": 0.0
        }

    # Count by severity
    high_count = sum(1 for s in vendor_signals if s.get("severity") == "high")
    medium_count = sum(1 for s in vendor_signals if s.get("severity") == "medium")
    low_count = sum(1 for s in vendor_signals if s.get("severity") == "low")

    # Count by type
    signal_types = defaultdict(int)
    for signal in vendor_signals:
        signal_type = signal.get("signal_type", "unknown")
        signal_types[signal_type] += 1

    # Find recent signals (last 30 days)
    today = date.today()
    recent_signals = []
    for signal in vendor_signals:
        detected_date_str = signal.get("detected_date")
        if detected_date_str:
            try:
                detected_date = datetime.fromisoformat(detected_date_str).date()
                days_ago = (today - detected_date).days
                if days_ago <= 30:
                    recent_signals.append(signal)
            except (ValueError, TypeError):
                pass

    # Find high priority signals (high severity)
    high_priority_signals = [s for s in vendor_signals if s.get("severity") == "high"]

    # Calculate impact score (weighted by severity and recency)
    # High = 100, Medium = 50, Low = 25
    # Recent (<=30 days) = 1.5x multiplier
    impact_score = 0.0
    for signal in vendor_signals:
        severity = signal.get("severity", "low")
        base_score = {"high": 100.0, "medium": 50.0, "low": 25.0}.get(severity, 0.0)

        # Check if recent
        detected_date_str = signal.get("detected_date")
        is_recent = False
        if detected_date_str:
            try:
                detected_date = datetime.fromisoformat(detected_date_str).date()
                days_ago = (today - detected_date).days
                is_recent = days_ago <= 30
            except (ValueError, TypeError):
                pass

        multiplier = 1.5 if is_recent else 1.0
        impact_score += base_score * multiplier

    # Normalize to 0-100 scale (assuming max 5 signals)
    max_possible_score = 5 * 100.0 * 1.5  # 5 high-severity recent signals
    normalized_score = min(100.0, (impact_score / max_possible_score) * 100.0) if max_possible_score > 0 else 0.0

    return {
        "total_signals": len(vendor_signals),
        "high_severity_count": high_count,
        "medium_severity_count": medium_count,
        "low_severity_count": low_count,
        "signal_types": dict(signal_types),
        "recent_signals": recent_signals,
        "high_priority_signals": high_priority_signals,
        "signal_impact_score": round(normalized_score, 2)
    }


def analyze_performance_metrics(
    vendor_id: str,
    vendor_performance: List[Dict[str, Any]]
) -> Dict[str, Any]:
    """
    Analyze vendor performance metrics.

    Args:
        vendor_id: Vendor to analyze
        vendor_performance: All vendor performance data

    Returns:
        Dictionary with performance analysis
        Structure:
        {
            "sla_compliance": 0.89,
            "sla_status": "below_threshold" | "meets_threshold" | "exceeds_threshold",
            "incident_count": 4,
            "incident_status": "elevated" | "normal" | "low",
            "response_time_avg_hours": 5.6,
            "response_time_status": "slow" | "acceptable" | "fast",
            "customer_satisfaction_score": 3.4,
            "satisfaction_status": "low" | "medium" | "high",
            "performance_score": 0-100,
            "performance_issues": [...]
        }
    """
    # Find performance data for this vendor
    perf_data = next(
        (p for p in vendor_performance if p.get("vendor_id") == vendor_id),
        None
    )

    if not perf_data:
        return {
            "sla_compliance": None,
            "sla_status": "no_data",
            "incident_count": None,
            "incident_status": "no_data",
            "response_time_avg_hours": None,
            "response_time_status": "no_data",
            "customer_satisfaction_score": None,
            "satisfaction_status": "no_data",
            "performance_score": 0.0,
            "performance_issues": []
        }

    sla_compliance = perf_data.get("sla_compliance")
    incident_count = perf_data.get("incident_count")
    response_time = perf_data.get("response_time_avg_hours")
    satisfaction = perf_data.get("customer_satisfaction_score")

    performance_issues = []

    # Analyze SLA compliance
    sla_status = "no_data"
    if sla_compliance is not None:
        if sla_compliance >= 0.95:
            sla_status = "exceeds_threshold"
        elif sla_compliance >= 0.90:
            sla_status = "meets_threshold"
        else:
            sla_status = "below_threshold"
            performance_issues.append(f"SLA compliance below threshold: {sla_compliance:.2%}")

    # Analyze incident count
    incident_status = "no_data"
    if incident_count is not None:
        if incident_count >= 5:
            incident_status = "elevated"
            performance_issues.append(f"Elevated incident count: {incident_count}")
        elif incident_count >= 2:
            incident_status = "normal"
        else:
            incident_status = "low"

    # Analyze response time
    response_time_status = "no_data"
    if response_time is not None:
        if response_time > 6.0:
            response_time_status = "slow"
            performance_issues.append(f"Slow response time: {response_time:.1f} hours")
        elif response_time > 4.0:
            response_time_status = "acceptable"
        else:
            response_time_status = "fast"

    # Analyze customer satisfaction
    satisfaction_status = "no_data"
    if satisfaction is not None:
        if satisfaction >= 4.0:
            satisfaction_status = "high"
        elif satisfaction >= 3.0:
            satisfaction_status = "medium"
        else:
            satisfaction_status = "low"
            performance_issues.append(f"Low customer satisfaction: {satisfaction:.1f}/5.0")

    # Calculate overall performance score (0-100)
    # Weighted: SLA 40%, Satisfaction 30%, Response Time 20%, Incidents 10%
    performance_score = 0.0
    components = 0

    if sla_compliance is not None:
        performance_score += sla_compliance * 100.0 * 0.40
        components += 0.40

    if satisfaction is not None:
        # Convert 1-5 scale to 0-100
        satisfaction_normalized = (satisfaction / 5.0) * 100.0
        performance_score += satisfaction_normalized * 0.30
        components += 0.30

    if response_time is not None:
        # Invert: faster is better (6 hours = 0, 0 hours = 100)
        response_time_normalized = max(0.0, min(100.0, (6.0 - response_time) / 6.0 * 100.0))
        performance_score += response_time_normalized * 0.20
        components += 0.20

    if incident_count is not None:
        # Invert: fewer incidents is better (5+ = 0, 0 = 100)
        incident_normalized = max(0.0, min(100.0, (5.0 - incident_count) / 5.0 * 100.0))
        performance_score += incident_normalized * 0.10
        components += 0.10

    # Normalize by components present
    if components > 0:
        performance_score = performance_score / components
    else:
        performance_score = 0.0

    return {
        "sla_compliance": sla_compliance,
        "sla_status": sla_status,
        "incident_count": incident_count,
        "incident_status": incident_status,
        "response_time_avg_hours": response_time,
        "response_time_status": response_time_status,
        "customer_satisfaction_score": satisfaction,
        "satisfaction_status": satisfaction_status,
        "performance_score": round(performance_score, 2),
        "performance_issues": performance_issues
    }


def detect_risk_drift(
    vendor_id: str,
    assessment_history: List[Dict[str, Any]],
    current_date: Optional[str] = None
) -> Optional[Dict[str, Any]]:
    """
    Detect risk drift by comparing current assessment to historical baseline.

    Args:
        vendor_id: Vendor to analyze
        assessment_history: All historical assessments
        current_date: Current assessment date (ISO format, defaults to today)

    Returns:
        Dictionary with drift analysis, or None if no history
        Structure:
        {
            "vendor_id": "VEND_001",
            "previous_score": 42,
            "current_score": None,  # Will be set during scoring phase
            "score_delta": None,
            "drift_direction": "increasing" | "decreasing" | "stable" | "unknown",
            "previous_assessment_date": "2025-10-01",
            "previous_risk_level": "medium",
            "previous_trigger": "scheduled_review",
            "drift_trigger": "external_signal" | "scheduled_review" | None
        }
    """
    # Filter history for this vendor
    vendor_history = [
        h for h in assessment_history
        if h.get("vendor_id") == vendor_id
    ]

    if not vendor_history:
        return None

    # Sort by date (most recent first)
    vendor_history.sort(
        key=lambda x: x.get("assessment_date", ""),
        reverse=True
    )

    # Get most recent historical assessment
    most_recent = vendor_history[0]

    previous_score = most_recent.get("risk_score")
    previous_date = most_recent.get("assessment_date")
    previous_level = most_recent.get("risk_level")
    previous_trigger = most_recent.get("trigger")

    # Check if there's a more recent signal-triggered assessment
    signal_triggered = None
    for assessment in vendor_history:
        if assessment.get("trigger") == "external_signal":
            signal_triggered = assessment
            break

    # Use signal-triggered if it's more recent
    if signal_triggered:
        signal_date = signal_triggered.get("assessment_date", "")
        if signal_date > previous_date:
            previous_score = signal_triggered.get("risk_score")
            previous_date = signal_date
            previous_level = signal_triggered.get("risk_level")
            previous_trigger = signal_triggered.get("trigger")

    return {
        "vendor_id": vendor_id,
        "previous_score": previous_score,
        "current_score": None,  # Will be set during scoring
        "score_delta": None,  # Will be calculated during scoring
        "drift_direction": "unknown",  # Will be determined during scoring
        "previous_assessment_date": previous_date,
        "previous_risk_level": previous_level,
        "previous_trigger": previous_trigger,
        "drift_trigger": previous_trigger
    }


def identify_risk_drivers(
    vendor_id: str,
    control_analysis: Dict[str, Dict[str, Any]],
    signal_analysis: Dict[str, Any],
    performance_analysis: Dict[str, Any],
    vendor_data: Dict[str, Any]
) -> List[str]:
    """
    Identify primary risk drivers by aggregating all analysis findings.

    Args:
        vendor_id: Vendor ID
        control_analysis: Control compliance analysis
        signal_analysis: External signal analysis
        performance_analysis: Performance metrics analysis
        vendor_data: Vendor metadata

    Returns:
        List of risk driver descriptions, prioritized by severity
    """
    drivers = []

    # Control-related drivers
    for domain_name, domain_data in control_analysis.items():
        status = domain_data.get("status")
        missing_controls = domain_data.get("missing_controls", [])
        expired_controls = [
            c.get("control") for c in domain_data.get("controls_detail", [])
            if c.get("status") == "expired"
        ]

        if expired_controls:
            for control in expired_controls:
                drivers.append(f"Expired {control} in {domain_name}")

        if missing_controls:
            for control in missing_controls:
                drivers.append(f"Missing {control} in {domain_name}")

        if status == "partial" and not expired_controls and not missing_controls:
            drivers.append(f"Partial compliance in {domain_name}")

    # Signal-related drivers
    high_priority_signals = signal_analysis.get("high_priority_signals", [])
    for signal in high_priority_signals:
        signal_type = signal.get("signal_type", "unknown")
        severity = signal.get("severity", "unknown")
        drivers.append(f"Recent {severity}-severity {signal_type.replace('_', ' ')}")

    # Performance-related drivers
    performance_issues = performance_analysis.get("performance_issues", [])
    drivers.extend(performance_issues)

    # Contract lifecycle drivers
    contract_status = vendor_data.get("contract_status")
    if contract_status == "renewal_pending":
        drivers.append("Contract renewal pending - requires review")
    elif contract_status == "onboarding":
        drivers.append("Vendor onboarding in progress - incomplete assessment")

    # Criticality-based drivers
    criticality = vendor_data.get("criticality")
    if criticality == "high":
        # High criticality vendors get extra scrutiny
        if len(drivers) > 0:
            drivers.append("High criticality vendor requires enhanced monitoring")

    return drivers


# Node

In [ ]:
def risk_analysis_node(state: ThirdPartyRiskOrchestratorState) -> Dict[str, Any]:
    """
    Risk Analysis Node: Orchestrate analyzing risk across all dimensions.

    Analyzes each vendor across:
    - Control compliance per risk domain
    - External risk signals
    - Performance metrics
    - Risk drift detection
    - Risk driver identification
    """
    from agents.third_party_risk_orchestrator.utilities.risk_analysis import (
        analyze_control_compliance,
        analyze_external_signals,
        analyze_performance_metrics,
        detect_risk_drift,
        identify_risk_drivers
    )

    errors = state.get("errors", [])
    third_parties = state.get("third_parties", [])
    vendor_controls = state.get("vendor_controls", [])
    external_signals = state.get("external_signals", [])
    vendor_performance = state.get("vendor_performance", [])
    assessment_history = state.get("assessment_history", [])
    risk_domains = state.get("risk_domains", [])
    risk_domain_lookup = state.get("risk_domain_lookup", {})
    vendor_lookup = state.get("vendor_lookup", {})

    if not third_parties:
        return {
            "errors": errors + ["risk_analysis_node: third_parties required"]
        }

    try:
        vendor_risk_analysis = {}
        risk_drift_detection = {}

        # Analyze each vendor
        for vendor in third_parties:
            vendor_id = vendor.get("vendor_id")
            if not vendor_id:
                continue

            # Analyze control compliance
            control_analysis = analyze_control_compliance(
                vendor_id,
                vendor_controls,
                risk_domains,
                risk_domain_lookup
            )

            # Analyze external signals
            signal_analysis = analyze_external_signals(
                vendor_id,
                external_signals
            )

            # Analyze performance metrics
            performance_analysis = analyze_performance_metrics(
                vendor_id,
                vendor_performance
            )

            # Detect risk drift
            drift_info = detect_risk_drift(
                vendor_id,
                assessment_history
            )
            if drift_info:
                risk_drift_detection[vendor_id] = drift_info

            # Identify risk drivers
            vendor_data = vendor_lookup.get(vendor_id, {})
            risk_drivers = identify_risk_drivers(
                vendor_id,
                control_analysis,
                signal_analysis,
                performance_analysis,
                vendor_data
            )

            # Combine all analysis
            vendor_risk_analysis[vendor_id] = {
                "vendor_id": vendor_id,
                "control_compliance": control_analysis,
                "external_signals": signal_analysis,
                "performance_metrics": performance_analysis,
                "risk_drivers": risk_drivers
            }

        return {
            "vendor_risk_analysis": vendor_risk_analysis,
            "risk_drift_detection": risk_drift_detection,
            "errors": errors
        }
    except Exception as e:
        return {
            "errors": errors + [f"risk_analysis_node: Unexpected error - {str(e)}"]
        }